## Create Capacity

Fabric Capacity is an Azure Resource.  In order to be able to assign capacities to a Power BI Workspace we must first create the capacity.  One important item to consider here is that when we create the capacity we assign the approved administrators for that capacity.  Later when we enumerate the available capacities in the Power BI API and assign them to workspaces, the user must have permissions on both the workspace and the

I was unable to find API documentation for this API.  I reverse engineered the API call by using the Azure Portal and choosing 'Download a Template' option during resoruce creation.

In [ ]:
pip install requests msal

In [ ]:
#For orgaization purposes I put notbooks in subfolders not the root of the proejct.aad_token
#This code adds the root directory of the project to the sys path so we can load class modules from the services folder
#I think this only needs to be run once, but including it for completeness.
import os, sys
projectRoot = os.path.abspath('.')
directory = os.path.dirname(projectRoot)
if not directory in sys.path: sys.path.append(directory)

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login.
#Note: for most of the other notebooks we're not passing in a scope, this is because the code in aadservice.property
#defaults the scope to a scope for the Power BI API.   Since we're calling the azure management API we need a different
#scope.
from services.aadservice import AadService
scope = 'https://management.azure.com/.default'
aad_token = AadService.get_access_token(scope)

headers =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + aad_token}

In [ ]:
#Note the Service Principal calling the rest API needs to be granted contributor access to whatever resource group you 
#use here.  Otherwise the service principal
resource_group = "cgmpbie"  

capacity_name = "cgmthisisatest"

#You need to set your own subscription id.
subscription_id = "7258b7d4-3429-4998-815b-8cd6954b7ef9"

#This can be an Email, or the object id (not application id) of the Service Prinicipal.
administrators = ['chmitch@microsoftanalytics.info','6709b293-4789-477f-aee2-607b7139e63c']

In [ ]:
import requests
import json

api_url = f'https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.Fabric/capacities/{capacity_name}?api-version=2022-07-01-preview'       

body = { 
            "type": "Microsoft.Fabric/capacities",
            "name": capacity_name,
            "location": "westus3",
            "sku": {
                "name": "F2",
                "tier": "Fabric"
            },
            "properties": {
                "administration": {
                    "members": administrators
                }
            }
        }

api_response = requests.put(api_url, headers=headers, data=json.dumps(body))

#error handling for create capacity
if api_response.status_code != 201 and api_response.status_code != 200:
    description = f'Error creating capacity:\n  -Status Code:\t{api_response.status_code}\n  -Reason:\t{api_response.reason}\n  -RequestId:\t{api_response.headers.get("RequestId")}\n  -Text:\t{api_response.text}'
    print(description)
else:
    api_response = json.loads(api_response.text)
    print(json.dumps(api_response,indent=2))